In [1]:
!pip install mysql-connector-python sqlalchemy pymysql  


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install pandas requests beautifulsoup4 yfinance tqdm



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
## HAM 1: CRAWL ALL

import requests
import pandas as pd
from io import StringIO
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import re

# --- Cấu hình ---
API_URL = 'https://inav.ice.com/api/1/tse/iopv/table?type=etf&language=en'
HEADERS = {'User-Agent': 'Mozilla/5.0', 'Referer': 'https://inav.ice.com'}
URL_TEMPLATE = "https://inav.ice.com/pcf-download/{code}.csv"
MAX_WORKERS = 12

# Hàm get_etf_codes
def get_etf_codes():
    """Lấy danh sách các ETF code hợp lệ từ API."""
    try:
        r = requests.get(API_URL, headers=HEADERS, timeout=10)
        r.raise_for_status()
        rows = r.json().get("rows", [])
        return [row["code"] for row in rows if row.get("code", "")]
    except requests.RequestException as e:
        print(f"⚠️ Lỗi khi lấy ETF codes từ API: {e}")
        return []

# Hàm parse_etf_csv
def parse_etf_csv(etf_code):
    """Tải và phân tích file CSV cho một ETF code, tính toán Value và thêm metadata."""
    url = URL_TEMPLATE.format(code=etf_code)
    shares_out = np.nan
    fund_date = np.nan
    
    try:
        r = requests.get(url, headers=HEADERS, timeout=10)
        r.raise_for_status()
        text = r.text
        
        # 1. Đọc metadata (dòng 2)
        meta = pd.read_csv(StringIO(text), skiprows=1, nrows=1, header=None)
        
        if meta.shape[1] >= 4: 
            shares_out = meta.iloc[0, 3]
        if meta.shape[1] >= 5: 
            fund_date = meta.iloc[0, 4]

        # 2. Đọc holdings (bắt đầu từ dòng 4)
        df = pd.read_csv(StringIO(text), skiprows=3)

        # 3. Đổi tên cột và chuẩn hóa
        df.rename(columns={
            "Shares Amount": "Shares_Amount",
            "Stock Price": "Stock_Price"
        }, inplace=True, errors='ignore')

        if 'Shares_Amount' not in df.columns or 'Stock_Price' not in df.columns:
             raise KeyError("Cột 'Shares_Amount' hoặc 'Stock_Price' không tìm thấy trong file CSV.")
        
        # 4. Chuyển đổi numeric
        df["Shares_Amount"] = pd.to_numeric(df["Shares_Amount"], errors="coerce")
        df["Stock_Price"] = pd.to_numeric(df["Stock_Price"], errors="coerce")
        df = df.dropna(subset=["Shares_Amount", "Stock_Price"])

        # 5. Tính toán và thêm metadata
        if df.empty:
            return None
            
        df["Value"] = df["Shares_Amount"] * df["Stock_Price"]
        df["ETF_Code"] = str(etf_code)
        df["Shares_Outstanding"] = shares_out
        df["Fund_Date"] = fund_date
        
        # 6. Sắp xếp thứ tự cột
        final_columns = ["ETF_Code", "Code", "Name", "ISIN", "Exchange", "Currency",
                         "Shares_Amount", "Stock_Price", "Value", "Shares_Outstanding", "Fund_Date"]

        for col in final_columns:
            if col not in df.columns:
                df[col] = np.nan 

        return df[final_columns]

    except requests.exceptions.HTTPError as e:
        print(f"⚠️ Lỗi HTTP cho {etf_code}: Không tìm thấy file CSV ({e})")
        return None
    except KeyError as e:
        print(f"⚠️ Lỗi Key/Cột cho {etf_code}: Cột dữ liệu chính bị thiếu. {e}")
        return None
    except Exception as e:
        print(f"⚠️ Lỗi phân tích cú pháp/khác cho {etf_code}: {e}")
        return None

# --- MAIN Execution
def main():
    """Chạy chương trình chính: lấy codes, tải, phân tích, hợp nhất và in ra toàn bộ DataFrame."""
    
    codes_list = get_etf_codes()
    if not codes_list:
        print("Không có ETF codes nào được tải xuống.")
        return None

    results_map = {}
    print(f"Đang xử lý {len(codes_list)} ETF codes...")

    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        futures = {ex.submit(parse_etf_csv, code): code for code in codes_list}
        
        for i, future in enumerate(as_completed(futures), 1):
            code = futures[future]
            df = future.result()
            
            if df is not None:
                results_map[code] = df
            
            print(f"[{i}/{len(codes_list)}] Đã xử lý code {code}.", end='\r', flush=True)

    print("\n--- Hoàn tất tải và phân tích ---")

    sorted_data = []
    for code in codes_list:
        if code in results_map:
            sorted_data.append(results_map[code])

    if sorted_data:
        df_final_detail = pd.concat(sorted_data, ignore_index=True)
        
    

        print(f"\n✅ Xong. Đã tạo DataFrame **df_final_detail** với {len(df_final_detail)} holdings.")
        print("\n--- df_final_detail (TOÀN BỘ NỘI DUNG) ---")
        
               
        return df_final_detail
        
    else:
        print("Không có DataFrame hợp lệ nào để hiển thị.")
        return None

if __name__ == "__main__":
    df_final_detail = main()


Đang xử lý 386 ETF codes...
⚠️ Lỗi Key/Cột cho 1326: Cột dữ liệu chính bị thiếu. "Cột 'Shares_Amount' hoặc 'Stock_Price' không tìm thấy trong file CSV."
⚠️ Lỗi Key/Cột cho 1308: Cột dữ liệu chính bị thiếu. "Cột 'Shares_Amount' hoặc 'Stock_Price' không tìm thấy trong file CSV."
⚠️ Lỗi Key/Cột cho 1305: Cột dữ liệu chính bị thiếu. "Cột 'Shares_Amount' hoặc 'Stock_Price' không tìm thấy trong file CSV."
⚠️ Lỗi Key/Cột cho 1322: Cột dữ liệu chính bị thiếu. "Cột 'Shares_Amount' hoặc 'Stock_Price' không tìm thấy trong file CSV."⚠️ Lỗi Key/Cột cho 1309: Cột dữ liệu chính bị thiếu. "Cột 'Shares_Amount' hoặc 'Stock_Price' không tìm thấy trong file CSV."

⚠️ Lỗi Key/Cột cho 1320: Cột dữ liệu chính bị thiếu. "Cột 'Shares_Amount' hoặc 'Stock_Price' không tìm thấy trong file CSV."
⚠️ Lỗi Key/Cột cho 1345: Cột dữ liệu chính bị thiếu. "Cột 'Shares_Amount' hoặc 'Stock_Price' không tìm thấy trong file CSV."
⚠️ Lỗi Key/Cột cho 133A: Cột dữ liệu chính bị thiếu. "Cột 'Shares_Amount' hoặc 'Stock_Price' khôn

In [4]:
## HAM 2: XU LI SAU CRAWL

df_final_detail.rename(columns={
    'Code' : 'code',
    'Name' : 'name',
    'Exchange' : 'exchange',
    'Currency' : 'currency',
    'Shares_Amount' : 'shares_amount',
    'Stock_Price' : 'stock_price',
    'Value' : 'value',
    'Shares_Outstanding' : 'shares_outstanding',
    'Fund_Date' : 'fund_date',
    'ISIN' : 'isin'})

df_final_detail = df_final_detail[
    [ "Code" , "Name", "Exchange", "Currency", "Shares_Amount", "Stock_Price" , "Value" , "ETF_Code" , "Fund_Date", "Shares_Outstanding" , "ISIN"]]

df_final_detail['Shares_Amount'] = df_final_detail['Shares_Amount'].astype(int)
df_final_detail['Shares_Outstanding'] = df_final_detail['Shares_Outstanding'].astype(int)
df_final_detail['Fund_Date'] = pd.to_datetime(df_final_detail['Fund_Date'], format='%Y%m%d')
df_final_detail.rename(columns={'ETF_Code' : 'etf_code'},inplace=True)

In [5]:
df_final_detail

,Code,Name,Exchange,Currency,Shares_Amount,Stock_Price,Value,etf_code,Fund_Date,Shares_Outstanding,ISIN
0,NaN,TOPIX 2512,OSE,JPY,16756,3328.0,5.576397e+07,1306,2025-12-04,8141966522,NaN
1,1301,KYOKUYO CO LTD,TSE,JPY,346100,4615.0,1.597252e+09,1306,2025-12-04,8141966522,JP3257200000
2,1332,NISSUI CORPORATION,TSE,JPY,7576000,1216.5,9.216204e+09,1306,2025-12-04,8141966522,JP3718800000
3,1333,MARUHA NICHIRO COR,TSE,JPY,1337800,3707.0,4.959225e+09,1306,2025-12-04,8141966522,JP3876600002
4,1375,YUKIGUNI FACTORY,TSE,JPY,703900,1040.0,7.320560e+08,1306,2025-12-04,8141966522,JP3947010009
...,...,...,...,...,...,...,...,...,...,...,...
45114,8750.0,DAI-ICHI LIFE HOLD,TSE,JPY,16500,1226.0,2.022961e+07,461A,2025-12-04,294900,JP3476480003
45115,8795.0,T&D HOLDINGS INC,TSE,JPY,4300,3464.0,1.489565e+07,461A,2025-12-04,294900,JP3539220008
45116,9101.0,NIPPON YUSEN KK,TSE,JPY,3800,4836.0,1.837736e+07,461A,2025-12-04,294900,JP3753000003
45117,9107.0,KAWASAKI KISEN KAI,TSE,JPY,10000,2063.0,2.063062e+07,461A,2025-12-04,294900,JP3223800008


In [6]:
df_final_detail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45119 entries, 0 to 45118
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Code                43772 non-null  object        
 1   Name                44973 non-null  object        
 2   Exchange            44217 non-null  object        
 3   Currency            45119 non-null  object        
 4   Shares_Amount       45119 non-null  int64         
 5   Stock_Price         45119 non-null  float64       
 6   Value               45119 non-null  float64       
 7   etf_code            45119 non-null  object        
 8   Fund_Date           45119 non-null  datetime64[ns]
 9   Shares_Outstanding  45119 non-null  int64         
 10  ISIN                44927 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(6)
memory usage: 3.8+ MB


In [7]:
import mysql.connector

In [8]:
## HAM 3: KET NOI MYSQL


import mysql.connector
cnx = mysql.connector.connect(
    user='root',               # đổi theo MySQL của bạn
    password='Phunghainam2004!',
    host='127.0.0.1',
    database='train'
)
cursor = cnx.cursor()

# --- Mapping DataFrame -> MySQL ---
# key = tên cột trong DataFrame, value = tên cột trong MySQL
col_mapping = {
    "Code" : "Code",
    "Name" : "Name",
    "Exchange" : "Exchange",
    "Currency" : "Currency",
    "Shares_Amount" :"Shares_Amount",
    "Stock_Price" : "Stock_Price",
    "Value" : "Value",
    "etf_code" : "etf_code",
    "Fund_Date" : "Fund_Date",
    "Shares_Outstanding" : "Shares_Outstanding",
    "ISIN" : "ISIN"    
}

# Lấy danh sách cột từ DataFrame và bảng
df_cols = list(col_mapping.keys())
table_cols = [col_mapping[c] for c in df_cols]

# --- Tạo câu SQL ---
col_str = ",".join([f"`{c}`" for c in table_cols])
placeholder_str = ",".join(["%s"] * len(df_cols))
sql = f"INSERT INTO `etf_asset_detail_train` ({col_str}) VALUES ({placeholder_str})"

# --- Insert từng dòng ---
for row in df_final_detail[df_cols].itertuples(index=False, name=None):
    row = tuple(None if pd.isna(x) else x for x in row)
    cursor.execute(sql, row)

cnx.commit()
cursor.close()
cnx.close()

print("✅ Đã insert df_final_detail vào etf_asset_detail_train thành công!")





